In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Imports

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.impute import IterativeImputer
from sklearn.experimental import enable_interative_imputer


ImportError: cannot import name 'IterativeImputer' from 'sklearn.impute' (C:\Users\ggomp\anaconda3\lib\site-packages\sklearn\impute\__init__.py)

In [6]:
df_train = pd.read_csv(r'./train.csv')
df_test = pd.read_csv(r'./test.csv')
df_sub = pd.read_csv(r'./gender_submission.csv')

display(df_train)
display(df_test)
# display(df_sub)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


Okay heres the order for the MVP training set:
1. Drop non-MVP columns
1. Drop NaN values
1. OHC Sex column
1. Split X and Y
1. Fit
1. Score

1. Drop non-MVP columns from both train and test sets

In [7]:
train_copy = df_train.copy()
train_mvp = train_copy.drop(['PassengerId','Name','Ticket','Cabin',"Embarked"], axis=1)
display(train_mvp)

test_copy = df_test.copy()
test_mvp = test_copy.drop(['PassengerId','Name','Ticket','Cabin',"Embarked"], axis=1)
display(test_mvp)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000
887,1,1,female,19.0,0,0,30.0000
888,0,3,female,NaN,1,2,23.4500
889,1,1,male,26.0,0,0,30.0000


,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,34.5,0,0,7.8292
1,3,female,47.0,1,0,7.0000
2,2,male,62.0,0,0,9.6875
3,3,male,27.0,0,0,8.6625
4,3,female,22.0,1,1,12.2875
...,...,...,...,...,...,...
413,3,male,NaN,0,0,8.0500
414,1,female,39.0,0,0,108.9000
415,3,male,38.5,0,0,7.2500
416,3,male,NaN,0,0,8.0500


2. Imputting NaN values

In [ ]:
imputer = IterativeImputer()
imputer.fit_transform(df_train)
display(df_train.head(50))

2. Drop NaN values


In [8]:
train_mvp = train_mvp.dropna(axis = 0)
train_mvp = train_mvp.reset_index(drop=True)
display(train_mvp)

test_mvp = test_mvp.dropna(axis = 0)
test_mvp = test_mvp.reset_index(drop=True)
display(test_mvp)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.0,1,0,7.2500
1,1,1,female,38.0,1,0,71.2833
2,1,3,female,26.0,0,0,7.9250
3,1,1,female,35.0,1,0,53.1000
4,0,3,male,35.0,0,0,8.0500
...,...,...,...,...,...,...,...
709,0,3,female,39.0,0,5,29.1250
710,0,2,male,27.0,0,0,13.0000
711,1,1,female,19.0,0,0,30.0000
712,1,1,male,26.0,0,0,30.0000


,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,male,34.5,0,0,7.8292
1,3,female,47.0,1,0,7.0000
2,2,male,62.0,0,0,9.6875
3,3,male,27.0,0,0,8.6625
4,3,female,22.0,1,1,12.2875
...,...,...,...,...,...,...
326,3,female,3.0,1,1,13.7750
327,1,female,37.0,1,0,90.0000
328,3,female,28.0,0,0,7.7750
329,1,female,39.0,0,0,108.9000


3. OHC Sex Column

In [9]:
train_mvp_c = train_mvp.copy()
train_mvp = pd.get_dummies(train_mvp_c,prefix=['Sex'], columns = ['Sex'], drop_first=True)
display(train_mvp)

test_mvp_c = test_mvp.copy()
test_mvp = pd.get_dummies(test_mvp_c,prefix=['Sex'], columns = ['Sex'], drop_first=True)
display(test_mvp)

,Survived,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,0,3,22.0,1,0,7.2500,1
1,1,1,38.0,1,0,71.2833,0
2,1,3,26.0,0,0,7.9250,0
3,1,1,35.0,1,0,53.1000,0
4,0,3,35.0,0,0,8.0500,1
...,...,...,...,...,...,...,...
709,0,3,39.0,0,5,29.1250,0
710,0,2,27.0,0,0,13.0000,1
711,1,1,19.0,0,0,30.0000,0
712,1,1,26.0,0,0,30.0000,1


,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,3,34.5,0,0,7.8292,1
1,3,47.0,1,0,7.0000,0
2,2,62.0,0,0,9.6875,1
3,3,27.0,0,0,8.6625,1
4,3,22.0,1,1,12.2875,0
...,...,...,...,...,...,...
326,3,3.0,1,1,13.7750,0
327,1,37.0,1,0,90.0000,0
328,3,28.0,0,0,7.7750,0
329,1,39.0,0,0,108.9000,0


3. Split X and Y

In [10]:
X, Y = train_mvp.iloc[:,1:], train_mvp.iloc[:, 0]

display(X)
display(Y)

,Pclass,Age,SibSp,Parch,Fare,Sex_male
0,3,22.0,1,0,7.2500,1
1,1,38.0,1,0,71.2833,0
2,3,26.0,0,0,7.9250,0
3,1,35.0,1,0,53.1000,0
4,3,35.0,0,0,8.0500,1
...,...,...,...,...,...,...
709,3,39.0,0,5,29.1250,0
710,2,27.0,0,0,13.0000,1
711,1,19.0,0,0,30.0000,0
712,1,26.0,0,0,30.0000,1


0      0
1      1
2      1
3      1
4      0
      ..
709    0
710    0
711    1
712    1
713    0
Name: Survived, Length: 714, dtype: int64

4. Create Pipeline

In [11]:
pipe = Pipeline([("scaling", StandardScaler()), ("model", KNeighborsClassifier(n_neighbors=5))])
# pipe.get_params()

5. Create GridSearchCV

In [19]:
model = GridSearchCV(estimator = pipe, 
                     param_grid = {'model__n_neighbors': [6,7,8,9,10,11,12,13,14, 18, 22]},
                     cv=10)

6. Model.fit

In [22]:
# %%timeit 

model.fit(X,Y)
df_results = pd.DataFrame(model.cv_results_)

highest_acc = df_results['mean_test_score'].max()
print(highest_acc*.792)
print()
display(df_results)


0.647869014084507



,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,split6_test_score,split7_test_score,split8_test_score,split9_test_score,mean_test_score,std_test_score,rank_test_score
0,0.008999,0.003684,0.008602,0.002337,6,{'model__n_neighbors': 6},0.763889,0.763889,0.750000,0.847222,0.816901,0.830986,0.845070,0.802817,0.901408,0.788732,0.811092,0.044501,2
1,0.005334,0.001342,0.004969,0.001187,7,{'model__n_neighbors': 7},0.750000,0.791667,0.763889,0.833333,0.788732,0.830986,0.845070,0.802817,0.887324,0.816901,0.811072,0.038466,3
2,0.004604,0.000658,0.004098,0.000299,8,{'model__n_neighbors': 8},0.736111,0.777778,0.750000,0.861111,0.816901,0.816901,0.845070,0.788732,0.901408,0.788732,0.808275,0.048235,7
3,0.004912,0.000545,0.004286,0.000629,9,{'model__n_neighbors': 9},0.750000,0.791667,0.763889,0.833333,0.788732,0.802817,0.845070,0.788732,0.873239,0.802817,0.804030,0.035337,10
4,0.004692,0.000637,0.004100,0.000542,10,{'model__n_neighbors': 10},0.763889,0.819444,0.763889,0.819444,0.802817,0.830986,0.859155,0.774648,0.859155,0.774648,0.806808,0.034861,9
5,0.005498,0.001017,0.004903,0.000941,11,{'model__n_neighbors': 11},0.777778,0.791667,0.750000,0.833333,0.802817,0.859155,0.830986,0.774648,0.859155,0.788732,0.806827,0.035277,8
6,0.005395,0.001347,0.004795,0.001164,12,{'model__n_neighbors': 12},0.777778,0.805556,0.763889,0.861111,0.859155,0.830986,0.845070,0.788732,0.859155,0.788732,0.818016,0.035442,1
7,0.005509,0.000921,0.004600,0.000801,13,{'model__n_neighbors': 13},0.763889,0.791667,0.763889,0.861111,0.816901,0.816901,0.845070,0.802817,0.845070,0.802817,0.811013,0.031383,4
8,0.005199,0.001079,0.004794,0.000881,14,{'model__n_neighbors': 14},0.791667,0.805556,0.763889,0.847222,0.845070,0.802817,0.830986,0.774648,0.845070,0.802817,0.810974,0.028480,5
9,0.005313,0.001008,0.005192,0.001006,18,{'model__n_neighbors': 18},0.777778,0.763889,0.763889,0.847222,0.802817,0.802817,0.802817,0.774648,0.873239,0.802817,0.801193,0.033699,11


In [ ]:
df_preds = pd.DataFrame(model.predict(test_mvp))
display(df_preds.head(50))

Notes from feature engineering:

Base MVP of just numeric columns = 58.6%
OHC of sex column: = 64.7%
Imputing Technique: 